In [18]:
import pandas as pd
import io
from typing import Union

In [20]:
df = pd.read_csv(r'C:\Users\arishneeln\Downloads\Egg Tray Press.csv', header=None) 
df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,HIOKI PW3360(Ver5.24),S/N.250711037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOLDER,25091600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WIRING,1P2W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OPERATION,RMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FREQUENCY,50Hz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def set_header_from_keyword(df: pd.DataFrame, keyword: str, col_index: int = 0) -> pd.DataFrame:
    """
    Restructures a DataFrame by finding a keyword in a specific column.

    This function searches for the first occurrence of a given keyword in the specified
    column. It then sets that row as the new header and returns a new DataFrame
    containing only the rows below the new header.

    Args:
        df (pd.DataFrame): The input DataFrame to process.
        keyword (str): The string to search for in the column to identify the header row.
        col_index (int, optional): The integer index of the column to search within.
                                   Defaults to 0 (the first column).

    Returns:
        pd.DataFrame: A new, cleaned DataFrame with the correct headers and data.
                      Returns an empty DataFrame if the keyword is not found.
    """
    # Ensure the input is a valid DataFrame
    if not isinstance(df, pd.DataFrame) or df.empty:
        print("Warning: Input is not a valid or non-empty DataFrame.")
        return pd.DataFrame()

    # Select the target column to search for the keyword
    search_column = df.iloc[:, col_index].astype(str)

    # Find the row index where the keyword first appears.
    # .str.contains() is more flexible than .eq() as it allows partial matches.
    # .idxmax() finds the index of the first 'True' value.
    try:
        header_row_index = search_column.str.contains(keyword, case=False, na=False).idxmax()
    except ValueError:
        print(f"Warning: Keyword '{keyword}' not found in column {col_index}.")
        return pd.DataFrame()

In [17]:
parameters = df.iloc[:14,:2].copy()
older_headers = parameters.columns.tolist()
new_headers = ["Parameter","Value"]
parameters.columns = new_headers
parameters.head(15)

,Parameter,Value
0,HIOKI PW3360(Ver5.24),S/N.250711037
1,FOLDER,25091600
2,WIRING,1P2W
3,OPERATION,RMS
4,FREQUENCY,50Hz
5,THD,NaN
6,INTERVAL,5min
7,U RANGE,600V
8,I RANGE,100A
9,SENSOR,CT9667(500A)


In [26]:
header_row_index = 16

# Create a new DataFrame starting from the data rows (row 18 onwards)
# .copy() is used to avoid SettingWithCopyWarning
data_df = df.iloc[header_row_index + 1:].copy()

# Set the new header using the values from row 17
data_df.columns = df.iloc[header_row_index]

# Reset the index to start from 0
data_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the newly created DataFrame
data_df.head()

16,Date,Etime,Status,Freq_Avg[Hz],U1_Avg[V],Ufnd1_Avg[V],Udeg1_Avg[deg],I1_Avg[A],Ifnd1_Avg[A],Ideg1_Avg[deg],...,WP+dem1[Wh],WP-dem1[Wh],WQLAGdem1[varh],WQLEADdem1[varh],Pdem+1[W],Pdem-1[W],QdemLAG1[var],QdemLEAD1[var],PFdem1,Pulse
0,16/09/2025 11:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/09/2025 11:25,0000:05:00,0.00,49.98,238.63,238.58,0.00,18.24,18.20,179.46,...,0.00,-360.70,0.20,-12.00,0.00,-4328.00,3.00,-144.00,0.00,0.00
2,16/09/2025 11:30,0000:10:00,0.00,50.01,238.35,238.30,0.00,23.24,23.22,179.51,...,0.00,-460.40,0.00,-13.40,0.00,-5525.00,0.00,-161.00,0.00,0.00
3,16/09/2025 11:35,0000:15:00,0.00,49.97,238.78,238.73,0.00,14.56,14.53,179.43,...,0.00,-288.00,0.10,-10.10,0.00,-3456.00,2.00,-121.00,0.00,0.00
4,16/09/2025 11:40,0000:20:00,0.00,49.96,238.44,238.39,0.00,22.84,22.81,179.50,...,0.00,-452.30,0.00,-13.90,0.00,-5427.00,0.00,-167.00,0.00,0.00


In [28]:
try:
    cols_to_check = data_df.drop('Date')
except KeyError:
    print("Warning: 'Date' column not found. Cannot perform cleaning operation.")
    # If 'Date' column doesn't exist, we can't proceed with this logic.
    # You might want to handle this case differently depending on your needs.
    cols_to_check = []

if cols_to_check.any():
    # Replace empty strings with NaN to make them detectable by dropna()
    data_df_cleaned = data_df_sliced.replace(r'^\s*$', pd.NA, regex=True)

    # Drop rows where all specified columns are NaN
    data_df_cleaned.dropna(subset=cols_to_check, how='all', inplace=True)

    # Display the shape before and after to see how many rows were removed
    print(f"Shape before cleaning: {data_df_sliced.shape}")
    print(f"Shape after cleaning:  {data_df_cleaned.shape}")

    # Display the head of the cleaned DataFrame
    data_df_cleaned.head()
else:
    # If cols_to_check is empty, just copy the dataframe
    data_df_cleaned = data_df_sliced.copy()
    print("Skipping cleaning as no columns to check were identified.")

AttributeError: 'list' object has no attribute 'any'